In [1]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import expr

# from pyspark.sql.types import StructType, StructField, StringType, IntegerType,DatetimeConverter,DateType,DoubleType,TimestampType,FloatType
# import json
# from time import sleep
# import concurrent.futures
# # Create a Spark session
# spark = SparkSession.builder \
#     .appName('KafkaStreamJoinExample') \
#     .getOrCreate()# <---- app name

# kafka_bootstrap_servers = 'localhost:9092'
# olist_orders1 = 'topic1'
# olist_customer2 = 'topic2'
# olist_order_items3 = 'seller1'
# olist_order_payments4 = 'newer'
# olist_order_reviews5 = 'topic5'




# # with concurrent.futures.ThreadPoolExecutor() as executor:
# #     # Submit tasks to create topics concurrently
# #     # Submit tasks to run producers concurrently
# #     futures = [executor.submit(run_producer, file, topic, producer) for file, topic in zip(files, kafka_topics)]
# #     concurrent.futures.wait(futures)

# # producer.close()


# # Read from Kafka topic 1
# olist_orders = spark.readStream \
#     .format('kafka') \
#     .option('kafka.bootstrap.servers', kafka_bootstrap_servers) \
#     .option('subscribe', olist_order_items3) \
#     .load() \
#     .selectExpr("CAST(value AS STRING)") \
#     .select(from_json(col("value"), csv_schema3).alias("data")) \
#     .select("data.*")

# olist_customer = spark.readStream \
#     .format('kafka') \
#     .option('kafka.bootstrap.servers', kafka_bootstrap_servers) \
#     .option('subscribe', olist_order_payments4) \
#     .load() \
#     .selectExpr("CAST(value AS STRING)") \
#     .select(from_json(col("value"), csv_schema4).alias("data")) \
#     .select("data.*")

# processed_df = olist_orders.select("product_id", "price")

# # Start the streaming query
# query1 = processed_df.writeStream \
#     .outputMode('append') \
#     .format('console') \
#     .start()

# processed_df1 = olist_customer.select("payment_type", "payment_installments")

# # Start the streaming query
# query2 = processed_df.writeStream \
#     .outputMode('append') \
#     .format('console') \
#     .start()

# query1.awaitTermination()

# query2.awaitTermination()

In [4]:
from kafka.admin import KafkaAdminClient, NewTopic
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col,expr,
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,DatetimeConverter,DateType,DoubleType,TimestampType,FloatType
import json
from time import sleep
import concurrent.futures

ImportError: cannot import name 'time'

In [ ]:
# Create a Spark session
spark = SparkSession.builder \
    .appName('KafkaConsumerVisualization') \
    .enableHiveSupport() \
    .getOrCreate()

In [ ]:
# .config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5") 

# spark.sparkContext.setLogLevel("ERROR")

# .config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.11-2.4.5") 
# spark-sql-kafka-0-10_2.12-3.1.1
# org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5
# org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1
# Kafka configuration
kafka_bootstrap_servers = '127.0.0.1:9092'
dash_topic='dash_topic'
kafka_topics = ['orders','customers','order_items', 'order_payments','order_reviews']

In [ ]:
# - olist_orders
#  0   order_id                       object        
#  1   customer_id                    object        
#  2   order_status                   object        
#  3   order_purchase_timestamp       datetime64[ns]
#  4   order_approved_at              datetime64[ns]
#  5   order_delivered_carrier_date   datetime64[ns]
#  6   order_delivered_customer_date  datetime64[ns]
#  7   order_estimated_delivery_date  datetime64[ns]
csv_schema1 = StructType([
    StructField("order_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("order_status", StringType(), True),
    StructField("order_purchase_timestamp", TimestampType(), True),
    StructField("order_approved_at", TimestampType(), True),
    StructField("order_delivered_carrier_date", TimestampType(), True),
    StructField("order_delivered_customer_date", TimestampType(), True),
    StructField("order_estimated_delivery_date", TimestampType(), True),
    StructField("current_time", TimestampType(), True)
])

# - olist_customer
#  0   customer_id               object
#  1   customer_unique_id        object
#  2   customer_zip_code_prefix  int64 
#  3   customer_city             object
#  4   customer_state            object
csv_schema2 = StructType([
    StructField("customer_id", StringType(), True),
    StructField("customer_unique_id", StringType(), True),
    StructField("customer_zip_code_prefix", IntegerType(), True),
    StructField("customer_city", StringType(), True),
    StructField("customer_state", StringType(), True),
    StructField("current_time", TimestampType(), True)
    
])

# - olist_order_items
# 0   order_id             object        
# 1   order_item_id        int64         
# 2   product_id           object        
# 3   seller_id            object        
# 4   shipping_limit_date  datetime64[ns]
# 5   price                float64       
# 6   freight_value        float64  
csv_schema3 = StructType([
    StructField("order_id", StringType(), True),
    StructField("order_item_id", IntegerType(), True),
    StructField("product_id", StringType(), True),
    StructField("seller_id", StringType(), True),
    StructField("shipping_limit_date", TimestampType(), True),
    StructField("price", FloatType(), True),
    StructField("freight_value", FloatType(), True),
    StructField("current_time", TimestampType(), True)
])

# - olist_order_payments
# 0   order_id              103886 non-null  object 
# 1   payment_sequential    103886 non-null  int64  
# 2   payment_type          103886 non-null  object 
# 3   payment_installments  103886 non-null  int64  
# 4   payment_value         103886 non-null  float64
csv_schema4 = StructType([
    StructField("order_id", StringType(), True),
    StructField("payment_sequential", IntegerType(), True),
    StructField("payment_type", StringType(), True),
    StructField("payment_installments", IntegerType(), True),
    StructField("payment_value", FloatType(), True),
    StructField("current_time", TimestampType(), True)
])

# - olist_order_reviews
# 0   review_id                99224 non-null  object
# 1   order_id                 99224 non-null  object
# 2   review_score             99224 non-null  int64 
# 3   review_comment_title     11568 non-null  object
# 4   review_comment_message   40977 non-null  object
# 5   review_creation_date     99224 non-null  object
# 6   review_answer_timestamp  99224 non-null  object
csv_schema5 = StructType([
    StructField("review_id", StringType(), True),
    StructField("order_id", StringType(), True),
    StructField("review_score", IntegerType(), True),
    StructField("review_creation_date", StringType(), True),
    StructField("review_answer_timestamp", StringType(), True),
    StructField("current_time", TimestampType(), True)
])

In [ ]:
# sellers
# 0   seller_id               3095 non-null   object
# 1   seller_zip_code_prefix  3095 non-null   int64 
# 2   seller_city             3095 non-null   object
# 3   seller_state            3095 non-null   object
sellers = StructType([
    StructField("seller_id", StringType(), True),
    StructField("seller_zip_code_prefix", IntegerType(), True),
    StructField("seller_city", StringType(), True),
    StructField("seller_state", StringType(), True)
])


# products
# 0   product_id                  32951 non-null  object 
# 1   product_category_name       32341 non-null  object 
# 2   product_name_lenght         32341 non-null  float64
# 3   product_description_lenght  32341 non-null  float64
# 4   product_photos_qty          32341 non-null  float64
# 5   product_weight_g            32949 non-null  float64
# 6   product_length_cm           32949 non-null  float64
# 7   product_height_cm           32949 non-null  float64
# 8   product_width_cm            32949 non-null  float64

products = StructType([
    StructField("product_id", StringType(), True),
    StructField("product_category_name", StringType(), True),
    StructField("product_name_lenght", FloatType(), True),
    StructField("product_description_lenght", FloatType(), True),
    StructField("product_photos_qty", FloatType(), True),
    StructField("product_weight_g", FloatType(), True),
    StructField("product_length_cm", FloatType(), True),
    StructField("product_height_cm", FloatType(), True),
    StructField("product_width_cm", FloatType(), True)
])


In [ ]:
# Read data from Kafka using Structured Streaming
orders = spark.readStream \
    .format('kafka') \
    .option('kafka.bootstrap.servers', '127.0.0.1:9092') \
    .option("subscribe", kafka_topics[0]) \
    .load() \
    .selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), csv_schema1).alias("data")) \
    .select("data.*")

customers = spark.readStream \
    .format('kafka') \
    .option('kafka.bootstrap.servers', '127.0.0.1:9092') \
    .option("subscribe", kafka_topics[1]) \
    .load() \
    .selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), csv_schema2).alias("data")) \
    .select("data.*")

order_items = spark.readStream \
    .format('kafka') \
    .option('kafka.bootstrap.servers', '127.0.0.1:9092') \
    .option("subscribe", kafka_topics[2]) \
    .load() \
    .selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), csv_schema3).alias("data")) \
    .select("data.*")

order_payments = spark.readStream \
    .format('kafka') \
    .option('kafka.bootstrap.servers', '127.0.0.1:9092') \
    .option("subscribe", kafka_topics[3]) \
    .load() \
    .selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), csv_schema4).alias("data")) \
    .select("data.*")


order_reviews = spark.readStream \
    .format('kafka') \
    .option('kafka.bootstrap.servers', '127.0.0.1:9092') \
    .option("subscribe", kafka_topics[4]) \
    .load() \
    .selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), csv_schema5).alias("data")) \
    .select("data.*")



In [ ]:
#joining ------------------------------------------------------------------------------------------------------------------
#static data using parquet
olist_products = spark.read.parquet("products.parquet")
olist_categname = spark.read.parquet("categname.parquet")
olist_sellers = spark.read.parquet("sellers.parquet")


#realtime data
# Windowing by timestamp with 1-minute intervals
from pyspark.sql.functions import timestamp_seconds


sdf.select(
   'name',
   timestamp_seconds(sdf.time).alias('time') #we need to add this after every 'data.*' in evert last select of df
                    |
                    |
).withWatermark('time', '10 minutes')
                    |        
                    +-----------------------+  # can do it like this if time stamp column is not present          
                                            |
sensor_window = sensor_df.withWatermark("current_time", "1 minute")\
  .groupBy("sensor_id").window("1 minute").agg(avg("value")) # we don't need groupby just simple join

user_window = user_df.withWatermark("timestamp", "1 minute")\
  .groupBy("user_id").window("1 minute").agg(count("action")) # we don't need groupby just simple join


joined_df = sensor_window.join(user_window, on=["timestamp", "user_id"], how="outer")




#-------------------------------------------------------------------------------------------------------------------

In [ ]:
def writeToDash(writeDF, _):
    writeDF.writeStream \
        .format('kafka') \
        .option('kafka.bootstrap.servers', '127.0.0.1:9092') \
        .option('topic', 'dash_topic') \
        .start()
    #.option('checkpointLocation', '/hdfs') \
    

def writeToHive1(writeDF, _):
    writeDF.write \
        .mode('append') \
        .saveAsTable('ml_table')

join_5.writeStream \
    .option("spark.cassandra.connection.host","127.0.0.1:9092")\
    .foreachBatch(writeToDash) \
    .foreachBatch(writeToHive1) \
    .outputMode("update") \
    .start()\
    .awaitTermination()





# add
# we need to add this after every 'data.*' in evert last select of df

b_orders = orders.withWatermark("current_time", "8 seconds")
b_customers = customers.withWatermark("current_time", "8 seconds")
b_order_items = order_items.withWatermark("current_time", "8 seconds")
b_order_payments = order_payments.withWatermark("current_time", "8 second")
b_order_reviews = order_reviews.withWatermark("current_time", "8 seconds")


join_1c=[
b_orders['customer_id']==b_customers['customer_id'],
b_orders['current_time']>=b_customers['current_time'],
b_orders['current_time']<=b_customers['current_time']+expr('interval 8 seconds')   
]


join_2c=[
join_1['order_id']==b_order_items['order_id'],
join_1['current_time']>=b_order_items['current_time'],
join_1['current_time']<=b_order_items['current_time']+expr('interval 8 seconds')   
]

join_3c=[
join_2['order_id']==b_order_payments['order_id'],
join_2['current_time']>=b_order_payments['current_time'],
join_2['current_time']<=b_order_payments['current_time']+expr('interval 8 seconds')   
]

join_4c=[
join_3['order_id']==b_order_reviews['order_id'],
join_3['current_time']>=b_order_reviews['current_time'],
join_3['current_time']<=b_order_reviews['current_time']+expr('interval 8 seconds')   
]

join_1=b_orders.join(b_customers,join_1c)
join_2=join_1.join(b_order_items,join_2c)
join_3=join_2.join(b_order_payments,join_3c)
join_4=join_3.join(b_order_reviews,join_4c)


join_5=join_4.join(olist_products,'product_id').join(olist_categname,'product_category_name').join(olist_sellers,'seller_id')




# data = olist_orders.merge(olist_customer, on="customer_id") \
# .merge(olist_order_items, on="order_id") \
# .merge(olist_products , on="product_id") \
# .merge(olist_categname, on="product_category_name") \
# .merge(olist_order_payments, on="order_id") \
# .merge(olist_sellers, on="seller_id") \
# .merge(olist_order_reviews, on="order_id")




# # Parse the JSON data based on the defined schema
# df_parsed = df.select(from_json(col("value"), csv_schema3).alias("data")) \
#     .select("data.*")


# # this function will feed the real time dash board
# def writeToDash(writeDF, _):
#     writeDF.writeStream \
#         .format('kafka') \
#         .option('kafka.bootstrap.servers', '127.0.0.1:9092') \
#         .option('topic', 'dash_topic') \
#         .start()
#     #.option('checkpointLocation', '/hdfs') \
    

# def writeToHive1(writeDF, _):
#     writeDF.write \
#         .mode('append') \
#         .saveAsTable('ml_table')

# df1.writeStream \
#     .option("spark.cassandra.connection.host","127.0.0.1:9092")\
#     .foreachBatch(writeToDash) \
#     .outputMode("update") \
#     .start()\
#     .awaitTermination()
    # it will direct all batch to streamlit
    # .foreachBatch(writeToHive2) \ # to save it in hive












# # Display the streaming data in Databricks
# try:
#     # This works in a Databricks notebook environment
#     df_parsed.writeStream \
#     .outputMode('append') \
#     .format('console') \
#     .start()
# except NameError:
#     pass  # Ignore if not running in Databricks

# Await termination (this will only be reached when the streaming query is manually stopped)
# query.awaitTermination()

# Stop the Spark session
# spark.stop()







#spark = SparkSession \
#    .builder \
#    .appName("Streaming from Kafka") \
#    .config("spark.streaming.stopGracefullyOnShutdown", True) \
#    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.11-2.4.5') \
#    .master("local[2]") \
#    .getOrCreate()

In [ ]:
from kafka.admin import KafkaAdminClient, NewTopic
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col,expr,
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,DatetimeConverter,DateType,DoubleType,TimestampType,FloatType
import json
from time import sleep
import concurrent.futures


spark = SparkSession.builder \
    .appName('KafkaConsumerVisualization') \
    .enableHiveSupport() \
    .getOrCreate()

kafka_bootstrap_servers = '127.0.0.1:9092'
dash_topic='dash_topic'
kafka_topics = ['orders','customers','order_items', 'order_payments','order_reviews']


# - olist_orders
#  0   order_id                       object        
#  1   customer_id                    object        
#  2   order_status                   object        
#  3   order_purchase_timestamp       datetime64[ns]
#  4   order_approved_at              datetime64[ns]
#  5   order_delivered_carrier_date   datetime64[ns]
#  6   order_delivered_customer_date  datetime64[ns]
#  7   order_estimated_delivery_date  datetime64[ns]
csv_schema1 = StructType([
    StructField("order_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("order_status", StringType(), True),
    StructField("order_purchase_timestamp", TimestampType(), True),
    StructField("order_approved_at", TimestampType(), True),
    StructField("order_delivered_carrier_date", TimestampType(), True),
    StructField("order_delivered_customer_date", TimestampType(), True),
    StructField("order_estimated_delivery_date", TimestampType(), True),
    StructField("current_time", TimestampType(), True)
])

# - olist_customer
#  0   customer_id               object
#  1   customer_unique_id        object
#  2   customer_zip_code_prefix  int64 
#  3   customer_city             object
#  4   customer_state            object
csv_schema2 = StructType([
    StructField("customer_id", StringType(), True),
    StructField("customer_unique_id", StringType(), True),
    StructField("customer_zip_code_prefix", IntegerType(), True),
    StructField("customer_city", StringType(), True),
    StructField("customer_state", StringType(), True),
    StructField("current_time", TimestampType(), True)
    
])

# - olist_order_items
# 0   order_id             object        
# 1   order_item_id        int64         
# 2   product_id           object        
# 3   seller_id            object        
# 4   shipping_limit_date  datetime64[ns]
# 5   price                float64       
# 6   freight_value        float64  
csv_schema3 = StructType([
    StructField("order_id", StringType(), True),
    StructField("order_item_id", IntegerType(), True),
    StructField("product_id", StringType(), True),
    StructField("seller_id", StringType(), True),
    StructField("shipping_limit_date", TimestampType(), True),
    StructField("price", FloatType(), True),
    StructField("freight_value", FloatType(), True),
    StructField("current_time", TimestampType(), True)
])

# - olist_order_payments
# 0   order_id              103886 non-null  object 
# 1   payment_sequential    103886 non-null  int64  
# 2   payment_type          103886 non-null  object 
# 3   payment_installments  103886 non-null  int64  
# 4   payment_value         103886 non-null  float64
csv_schema4 = StructType([
    StructField("order_id", StringType(), True),
    StructField("payment_sequential", IntegerType(), True),
    StructField("payment_type", StringType(), True),
    StructField("payment_installments", IntegerType(), True),
    StructField("payment_value", FloatType(), True),
    StructField("current_time", TimestampType(), True)
])

# - olist_order_reviews
# 0   review_id                99224 non-null  object
# 1   order_id                 99224 non-null  object
# 2   review_score             99224 non-null  int64 
# 3   review_comment_title     11568 non-null  object
# 4   review_comment_message   40977 non-null  object
# 5   review_creation_date     99224 non-null  object
# 6   review_answer_timestamp  99224 non-null  object
csv_schema5 = StructType([
    StructField("review_id", StringType(), True),
    StructField("order_id", StringType(), True),
    StructField("review_score", IntegerType(), True),
    StructField("review_creation_date", StringType(), True),
    StructField("review_answer_timestamp", StringType(), True),
    StructField("current_time", TimestampType(), True)
])


# Read data from Kafka using Structured Streaming
orders = spark.readStream \
    .format('kafka') \
    .option('kafka.bootstrap.servers', '127.0.0.1:9092') \
    .option("subscribe", kafka_topics[0]) \
    .load() \
    .selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), csv_schema1).alias("data")) \
    .select("data.*")

customers = spark.readStream \
    .format('kafka') \
    .option('kafka.bootstrap.servers', '127.0.0.1:9092') \
    .option("subscribe", kafka_topics[1]) \
    .load() \
    .selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), csv_schema2).alias("data")) \
    .select("data.*")

order_items = spark.readStream \
    .format('kafka') \
    .option('kafka.bootstrap.servers', '127.0.0.1:9092') \
    .option("subscribe", kafka_topics[2]) \
    .load() \
    .selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), csv_schema3).alias("data")) \
    .select("data.*")

order_payments = spark.readStream \
    .format('kafka') \
    .option('kafka.bootstrap.servers', '127.0.0.1:9092') \
    .option("subscribe", kafka_topics[3]) \
    .load() \
    .selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), csv_schema4).alias("data")) \
    .select("data.*")


order_reviews = spark.readStream \
    .format('kafka') \
    .option('kafka.bootstrap.servers', '127.0.0.1:9092') \
    .option("subscribe", kafka_topics[4]) \
    .load() \
    .selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), csv_schema5).alias("data")) \
    .select("data.*")

olist_products = spark.read.parquet("products.parquet") #<-- to create throught spark to parquet to hdfs
olist_categname = spark.read.parquet("categname.parquet") #<-- to create
olist_sellers = spark.read.parquet("sellers.parquet")#<-- to create


b_orders = orders.withWatermark("current_time", "8 seconds")
b_customers = customers.withWatermark("current_time", "8 seconds")
b_order_items = order_items.withWatermark("current_time", "8 seconds")
b_order_payments = order_payments.withWatermark("current_time", "8 second")
b_order_reviews = order_reviews.withWatermark("current_time", "8 seconds")


join_1c=[
b_orders['customer_id']==b_customers['customer_id'],
b_orders['current_time']>=b_customers['current_time'],
b_orders['current_time']<=b_customers['current_time']+expr('interval 8 seconds')   
]


join_2c=[
join_1['order_id']==b_order_items['order_id'],
join_1['current_time']>=b_order_items['current_time'],
join_1['current_time']<=b_order_items['current_time']+expr('interval 8 seconds')   
]

join_3c=[
join_2['order_id']==b_order_payments['order_id'],
join_2['current_time']>=b_order_payments['current_time'],
join_2['current_time']<=b_order_payments['current_time']+expr('interval 8 seconds')   
]

join_4c=[
join_3['order_id']==b_order_reviews['order_id'],
join_3['current_time']>=b_order_reviews['current_time'],
join_3['current_time']<=b_order_reviews['current_time']+expr('interval 8 seconds')   
]

join_1=b_orders.join(b_customers,join_1c)
join_2=join_1.join(b_order_items,join_2c)
join_3=join_2.join(b_order_payments,join_3c)
join_4=join_3.join(b_order_reviews,join_4c)


join_5=join_4.join(olist_products,'product_id').join(olist_categname,'product_category_name').join(olist_sellers,'seller_id')



def writeToDash(writeDF, _):
    writeDF.writeStream \
        .format('kafka') \
        .option('kafka.bootstrap.servers', '127.0.0.1:9092') \
        .option('topic', 'dash_topic') \
        .start()
    #.option('checkpointLocation', '/hdfs') \
    

def writeToHive1(writeDF, _):
    writeDF.write \
        .mode('append') \
        .saveAsTable('ml_table')

join_5.writeStream \
    .option("spark.cassandra.connection.host","127.0.0.1:9092")\
    .foreachBatch(writeToDash) \
    .foreachBatch(writeToHive1) \
    .outputMode("update") \
    .start()\
    .awaitTermination()